# Using the Model

> This notebook explains how one can load a SysML v2 model and interact with the data

In [1]:
from pathlib import Path

import pymbe.api as pm

In [2]:
fixtures_folder = Path(pm.__file__).parent / "../../tests/fixtures/"

In [3]:
model = pm.Model.load_from_file(fixtures_folder / "Kerbal.json")
# model = pm.Model.load_from_file(list(fixtures_folder.glob("*.json"))[6])

model

<SysML v2 Model (C:\sandboxes\pymbe\tests\fixtures\Kerbal.json)>

you can save the model back to a file

In [4]:
model.save_to_file("New Kerbal.json")

and load it back...

In [5]:
new_model = pm.Model.load_from_file("New Kerbal.json")

assert len(new_model.elements) > 0
assert len(new_model.elements) == len(model.elements)

the model can own elements (`.ownedElement`) and relationships (`.ownedRelationship`), and an element can be retrieved by name

model.ownedElement[0].ownedElement["14a-Language Extensions"].ownedElement["User Defined Extensions"].ownedElement["ClassificationLevel"].data

In [7]:
kerbal = model.ownedElement["Kerbal"]

assert kerbal.name == "Kerbal"

kerbal

<Kerbal «Package»>

this can be done sequentially through the chain of element ownership

In [8]:
kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0]["value"]

0.125

In [9]:
assert kerbal.ownedElement["Parts Library"].ownedElement["FL-T200 Fuel Tank"].ownedElement["Empty Mass"].ownedElement[0].value == 0.125

all the model elements are stored under `model.elements` and can be retrieved by their SysML identifier

In [10]:
a_return_parameter_membership = model.elements["20511137-d751-4baf-ac4b-994a8e65ad3d"]
a_return_parameter_membership

<ReturnParameterMembership(<efeff587-3477-4eaf-8c59-fce1471ae1dc «LiteralReal»> ←→ <$result «Feature»>)>

and their properties are resolved to the appropriate element

In [11]:
a_return_parameter_membership.relatedElement[0].value

1.5

because of the nature of the data, there are multiple ways to refer to the same element, and the interactions allow for that

In [12]:
assert a_return_parameter_membership.target.reverseReturnParameterMembership == a_return_parameter_membership.relatedElement[0]

In [13]:
assert a_return_parameter_membership._id == a_return_parameter_membership._data["@id"]

In [14]:
model.elements["0b6a7f54-f8bf-4a16-b5aa-a309321256c1"].source.throughSuperclassing.name == "Kerbal Rocket Part"

True

we can also explore all the relationships in a given element

a `through<Relationship Metatype>` is attached to the source, while a `reverse<Relationship Metatype>` is attached to the targer

In [15]:
an_element = a_return_parameter_membership.relatedElement[1]

a_return_parameter_membership.target.relationships

{'reverseReturnParameterMembership': <efeff587-3477-4eaf-8c59-fce1471ae1dc «LiteralReal»>}

we can also see all the relationships

In [16]:
from pprint import pprint


for element_id, element in model.elements.items():
    name = element._data.get("qualifiedName") or element_id
    relationships = element.relationships

    if not relationships:
        continue
    print("\n", name)
    pprint(relationships)


 Kerbal::'Rocket Building'::'Fuel Tank Section'::'Full Mass'
{'reverseFeatureMembership': <Fuel Tank Section «PartDefinition»>,
 'reverseMembership': <2665fb1b-1f12-4f13-a977-0f060915773e «FeatureReferenceExpression»>,
 'reverseRedefinition': [<Full Mass «AttributeUsage»>,
                         <Full Mass «AttributeUsage»>],
 'throughFeatureTyping': <Real «DataType»>}

 Kerbal::'Rocket Building'::'Solid Booster'::'Full Mass'
{'reverseFeatureMembership': <Solid Booster «PartDefinition»>,
 'reverseRedefinition': [<Full Mass «AttributeUsage»>,
                         <Full Mass «AttributeUsage»>],
 'throughFeatureTyping': <Real «DataType»>}

 Kerbal::'Parts Library'::'RT-10 \"Hammer\" Solid Fuel Booster'::'Empty Mass'
{'reverseFeatureMembership': <RT-10 "Hammer" Solid Fuel Booster «PartDefinition»>,
 'throughFeatureTyping': <Real «DataType»>,
 'throughFeatureValue': <8484ac83-79fe-4181-a722-ced27c1a9dde «LiteralReal»>,
 'throughRedefinition': <Empty Mass «AttributeUsage»>}

 Kerbal::